# Data Processing
Designed by Tayven Stover.

In [17]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
import json

TRAINING_DATA_FOLDER = '../data/'

In [18]:
def select_useful_columns(df, columns_to_keep):
    columns = [col for col in df.columns if col not in columns_to_keep]

    df = df.drop(columns=columns)

    return df

def get_mean_location(df):
    mean_location = df[['latitude', 'longitude']].mean()

    # Convert to list
    mean_location = mean_location.to_list()

    return mean_location

def run_length_encoding(df, column_name):
    # Apply RLE: Get run lengths and values
    n = len(df)
    y = np.array(df[f'{column_name}_is_nan'])
    starts = np.r_[0, np.flatnonzero(y[1:] != y[:-1]) + 1]
    lengths = np.diff(np.r_[starts, n])
    values = y[starts]
    return starts, lengths, values

def mark_large_gaps(df, column_name, gap_threshold=5):
    # Ensure the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"The column '{column_name}' does not exist in the DataFrame.")

     # Flag rows that are NaN
    df[f'{column_name}_is_nan'] = df[column_name].isna().astype(int)
    
    # Apply RLE on the column
    starts, lengths, values = run_length_encoding(df, column_name)
    
    # Initialize the validity column with ones
    df[f'{column_name}_valid_sequence'] = 1
    
    # Identify the start positions of large NaN gaps and their lengths
    large_gaps = (values == 1) & (lengths > gap_threshold)

    # Set the validity of sequences following large gaps to 0
    for start, length in zip(starts[large_gaps], lengths[large_gaps]):
        df.loc[start:start+length-1, f'{column_name}_valid_sequence'] = 0
    
    df.drop([f'{column_name}_is_nan'], axis=1, inplace=True)
    
    return df

def interpolate_missing_values(df, columns_to_interpolate):
    # Specify validation columns
    validation_columns = [f'{column}_valid_sequence' for column in columns_to_interpolate]
    
    # Determine rows eligible for interpolation
    # Only interpolate rows where all associated validation columns are 1
    df['interpolate_flag'] = df[validation_columns].all(axis=1)

    # Iterate over each column that needs interpolation
    for column in columns_to_interpolate:
        if column in df.columns:
            # Use 'mask' to isolate parts of the column that should be interpolated
            # This will replace values where interpolate_flag is 0 with NaN, which are then not interpolated
            mask = df['interpolate_flag'] == 1
            # Temporarily store the original data
            original_data = df[column].copy()
            # Replace data not to be interpolated with NaN
            df.loc[~mask, column] = np.nan
            # Interpolate missing (NaN) values only where the mask is True
            df[column] = df[column].interpolate(method='linear', limit_direction='both')
            # Replace the NaN values back with the original data to avoid affecting non-interpolated parts
            df.loc[~mask, column] = original_data[~mask]

    return df

def encode_categories(category_dataframe):
    encoder = OneHotEncoder(sparse_output=False)

    # Fit the encoder
    encoder.fit(category_dataframe)

    # Transform the data
    encoded_data = encoder.transform(category_dataframe)

    return encoded_data

def process_category(category_data):
    # Print unique values in the category data
    category_keys = np.unique(category_data)

    category_df = pd.DataFrame(category_data)

    data_array = encode_categories(category_df)

    feature_len = np.shape(data_array)[-1]
    reshaped_data = data_array.reshape(-1, feature_len)

    category_values = np.unique(reshaped_data, axis=0)
    
    # Convert from np.int32 to int
    if type(category_keys[0]) == np.int32:
        category_dict = {int(category_keys[i]): category_values[i].tolist() for i in range(len(category_keys))}
        
    else:
        # Compile a dict
        category_dict = {category_keys[i]: category_values[i].tolist() for i in range(len(category_keys))}

    return reshaped_data, category_dict

def sequence_categorical_data(category_data):
    for i in range((int(len(category_data) / 24))):
        start_idx = i * 24
        end_idx = start_idx + 24
        block = category_data[start_idx:end_idx]
        
        # Find the first non-empty string in the block
        first_non_empty_string = next((x[0] for x in block if x[0] != ''), None)
        
        if first_non_empty_string:
            # Replace all values in the block with the first non-empty string
            category_data[start_idx:end_idx] = first_non_empty_string

    # Remove 23 values from every 24 values this just saves memory
    category_data = category_data[::24]

    return category_data

# Takes an np array of dates (y, m, d) and returns two arrays of months and days (x, 1)
# TODO update to also return start hour of each sequence!!!
def process_date_data(date_data):
    # Convert list to df with date col
    full_date_df = pd.DataFrame(date_data, columns=['Date'])
    full_date_df['Date'] = pd.to_datetime(full_date_df['Date'], format="%Y-%m-%d")

    full_date_df['Month'] = full_date_df['Date'].dt.strftime("%b")
    full_date_df['Day'] = full_date_df['Date'].dt.day
    full_date_df['Hour'] = full_date_df['Date'].dt.hour

    # Extract the Month and Day columns as NumPy arrays
    month_array = full_date_df['Month'].to_numpy().reshape(-1, 1)
    day_array = full_date_df['Day'].to_numpy().reshape(-1, 1)
    hour_array = full_date_df['Hour'].to_numpy().reshape(-1, 1)
    
    return month_array, day_array, hour_array

def cleanse_df_blocks(df, block_size = 24):
    num_rows = len(df)
    num_full_blocks = num_rows // block_size
    new_num_rows = num_full_blocks * block_size

    # Slice the DataFrame to keep only the rows up to 'new_num_rows'
    df = df.iloc[:new_num_rows]

    # Step 2: Reset the index to ensure it starts from 0 and is sequential
    df = df.reset_index(drop=True)

    # Step 3: Assign block numbers to each row
    df['block'] = df.index // block_size

    # Step 4: Remove blocks that contain any NaN values
    # This function returns True if the block has no NaN values, so it will be kept
    df_cleaned = df.groupby('block').filter(lambda x: not x.isnull().values.any())

    # Step 5: Drop the 'block' column if it's no longer needed
    df_cleaned = df_cleaned.drop(columns=['block'])

    return df_cleaned

In [19]:
def create_sequences(df, x_columns, y_columns, time_steps=24):
    x_sequence = []
    y_sequence = []
    date_sequence = []
    used_indices = []  # List to hold indices used in sequences

    # Iterate through the DataFrame to form sequences
    for i in range(0, len(df) - time_steps + 1, time_steps):

        temp_df = df.iloc[i:i + time_steps]

        # Check if all entries in the sequence have interpolate_flag set to TRUE and timestamps are consecutive
        if temp_df['interpolate_flag'].all() and \
           (temp_df['date'].iloc[-1] - temp_df['date'].iloc[0]).total_seconds() == (time_steps - 1) * 3600:
            # Append the sequences to the lists as NumPy arrays
            x_sequence.append(temp_df[x_columns].to_numpy())
            y_sequence.append(temp_df[y_columns].to_numpy())
            date_sequence.append(temp_df['date'].to_numpy())
            used_indices.extend(range(i, i + time_steps))  # Add all indices in this sequence to the list

    # Convert lists to numpy arrays
    x_sequence = np.array(x_sequence)
    y_sequence = np.array(y_sequence)
    date_sequence = np.array(date_sequence)
    # Find indices that are not part of any sequence
    unused_indices = set(range(len(df))) - set(used_indices)
    unused_rows = df.iloc[list(unused_indices)].sort_index()

    return x_sequence, y_sequence, date_sequence, unused_rows


def shuffle_data(*arrays, random_state=42):
    np.random.seed(random_state)
    indices = np.arange(len(arrays[0]))
    np.random.shuffle(indices)
    return [np.array(arr)[indices] for arr in arrays]

def split_by_proportion(arrays, split_ratio):
    split_index = int(len(arrays[0]) * split_ratio)
    return [arr[:split_index] for arr in arrays], [arr[split_index:] for arr in arrays]

def split_data(X, y, latitude, longitude, month, day, hour, test_size=0.2, validation_size=0.15, random_state=42):
    # Shuffle all data arrays consistently
    X, y, latitude, longitude, month, day, hour = shuffle_data(X, y, latitude, longitude, month, day, hour, random_state=random_state)
    
    # Split for test set
    train_val_data, test_data = split_by_proportion([X, y, latitude, longitude, month, day, hour], 1 - test_size)
    X_train_val, y_train_val, lat_train_val, lon_train_val, month_train_val, day_train_val, hour_train_val = train_val_data
    X_test, y_test, lat_test, lon_test, month_test, day_test, hour_test = test_data

    # Split train and validation sets
    train_data, val_data = split_by_proportion([X_train_val, y_train_val, lat_train_val, lon_train_val, month_train_val, day_train_val, hour_train_val], 1 - validation_size)
    X_train, y_train, lat_train, lon_train, month_train, day_train, hour_train = train_data
    X_validate, y_validate, lat_validate, lon_validate, month_validate, day_validate, hour_validate = val_data

    return {
        "train": (X_train, y_train, lat_train, lon_train, month_train, day_train, hour_train),
        "validate": (X_validate, y_validate, lat_validate, lon_validate, month_validate, day_validate, hour_validate),
        "test": (X_test, y_test, lat_test, lon_test, month_test, day_test, hour_test),
    }

def scale_data(x_train, x_test, x_validate, y_train, y_test, y_validate):
    # Initialize separate scalers for X and Y
    x_scaler = MinMaxScaler(feature_range=(0, 1))
    y_scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit X scaler on the training data
    x_scaler.fit(np.concatenate(x_train))

    # Scale X data
    x_train_scaled = [x_scaler.transform(df) for df in x_train]
    x_test_scaled = [x_scaler.transform(df) for df in x_test]
    x_validate_scaled = [x_scaler.transform(df) for df in x_validate]

    # Reshape Y data for scaling (flattening sample and time dimensions)
    y_train_flat = np.array(y_train).reshape(-1, 1)
    y_test_flat = np.array(y_test).reshape(-1, 1)
    y_validate_flat = np.array(y_validate).reshape(-1, 1)

    # Fit Y scaler on the flattened training labels
    y_scaler.fit(y_train_flat)

    # Scale Y data and reshape back to (n_samp, 24, 1)
    y_train_scaled = y_scaler.transform(y_train_flat).reshape(-1, 24, 1)
    y_test_scaled = y_scaler.transform(y_test_flat).reshape(-1, 24, 1)
    y_validate_scaled = y_scaler.transform(y_validate_flat).reshape(-1, 24, 1)

    return x_train_scaled, x_test_scaled, x_validate_scaled, y_train_scaled, y_test_scaled, y_validate_scaled



def save_data(directory, data, names):
    types = ['train', 'test', 'validate']

    i = 0

    for name in names:
        for data_type in types:
            file_path = os.path.join(directory, f'{name}_{data_type}.npy')

            np.save(file_path, data[i])

            i += 1
    
def initialize_processing(file, COLUMNS_TO_KEEP):    
    sensor_readings_df = pd.read_csv(file) # Load in file.

    # Step 0: Drop unnecessary columns and columns with a date not falling on the hour
    sensor_readings_df = select_useful_columns(sensor_readings_df, COLUMNS_TO_KEEP)
    sensor_readings_df = sensor_readings_df[sensor_readings_df['date'].str.endswith(':00:00')]

    # Step 0.1: Ensure location remains the same
    mean_location = get_mean_location(sensor_readings_df)
    sensor_readings_df['latitude'] = mean_location[0]
    sensor_readings_df['longitude'] = mean_location[1]

    # Step 0.2: Remove all impossible temperature values (less than -2) and (greater than 50)
    sensor_readings_df = sensor_readings_df[sensor_readings_df['sea_water_temperature'] >= 0]
    sensor_readings_df = sensor_readings_df[sensor_readings_df['sea_water_temperature'] <= 50]

    # Step 1: Add is_original column and set to 1
    sensor_readings_df['is_original'] = 1
 
    # Step 2: Convert date column to date type
    sensor_readings_df['date'] = pd.to_datetime(sensor_readings_df['date'], format='%Y-%m-%d %H:%M:%S')
    sensor_readings_df = sensor_readings_df.sort_values(by='date').reset_index(drop=True)
 
    # Step 3: Create a complete date range
    full_range = pd.date_range(start=sensor_readings_df['date'].min(), end=sensor_readings_df['date'].max(), freq='h')
    full_df = pd.DataFrame(full_range, columns=['date'])
 
    # Step 4: Merge with the original dataframe
    merge_df = pd.merge(full_df, sensor_readings_df, on='date', how='left')
    merge_df['is_original'] = merge_df['is_original'].fillna(0) # Set is_original to 0 for new rows
 
    # Step 5: Fill other columns for new rows with NaN
    for col in set(merge_df.columns) - {'date', 'is_original'}:
        if col not in sensor_readings_df:
            continue
        merge_df[col] = merge_df[col].where(merge_df['is_original'] == 1)
    
    # Step 7: Mark large gaps in columns - latitude, longitude, sea_water_temperature, and date
    # Step 7.1: latitude
    lat_mark_df = mark_large_gaps(merge_df, 'latitude', gap_threshold=5)

    # Step 7.2: longitude
    long_mark_df = mark_large_gaps(lat_mark_df, 'longitude', gap_threshold=5)

    # Step 7.3: Sample measurement
    sea_temp_mark_df = mark_large_gaps(long_mark_df, 'sea_water_temperature', gap_threshold=5)

    # Step 7.4: Date
    date_mark_df = mark_large_gaps(sea_temp_mark_df, 'date', gap_threshold=5)

    # Step 8: Interpolate small gaps in the columns using linear interpolation
    interpolate_df = interpolate_missing_values(date_mark_df, COLUMNS_TO_KEEP)

    # Step 9: remove all 24 row blocks with NaN values
    # IK not the best but were on a time crunch
    # Hello tech debt!
    interpolate_df = cleanse_df_blocks(interpolate_df)

    # Step 10: add a column for future temperature data
    interpolate_df['future_temp'] = interpolate_df['sea_water_temperature'].shift(-24)
    interpolate_df.drop(interpolate_df.tail(24).index, inplace=True)
    
    return interpolate_df

In [20]:
# Prepare data
RAW_DATA_FOLDER = f'{TRAINING_DATA_FOLDER}/raw/'
MAIN_FOLDER = os.listdir(RAW_DATA_FOLDER) # Get all raw data files

COVARIATE_COLUMNS = ['latitude', 'longitude', 'date', 'sea_water_temperature']
TARGET_COLUMN = 'future_temp'

time_steps = 24
feature_list = []
target_list = []
date_list = []

file_count = len(MAIN_FOLDER)

# Create an empty dataframe
df = pd.DataFrame()

# Loop through the files and add to df
for i in tqdm(range(file_count), desc='Gathering and sequencing data', unit='files'):
    file = f'{RAW_DATA_FOLDER}{MAIN_FOLDER[i]}'

    if file.endswith('.csv'):
        df = initialize_processing(file, COVARIATE_COLUMNS)

        x_sequence, y_sequence, date_sequence, unused_rows = create_sequences(df, COVARIATE_COLUMNS, TARGET_COLUMN, time_steps)

        # Append to overall array
        feature_list.extend(x_sequence)
        target_list.extend(y_sequence)
        date_list.extend(date_sequence)

# Save feature, target, and date data as .npy incase memory runs out
np.save(f'{TRAINING_DATA_FOLDER}/feature_data.npy', feature_list)
np.save(f'{TRAINING_DATA_FOLDER}/target_data.npy', target_list)
np.save(f'{TRAINING_DATA_FOLDER}/date_data.npy', date_list)

Gathering and sequencing data:   0%|          | 0/53 [00:00<?, ?files/s]

In [21]:
print('Extracting and encoding categories')
# Process all categorical data
# Date first
date_data = np.array(date_list).reshape(-1, 1)
date_data = sequence_categorical_data(date_data)

month_data, day_data, hour_data = process_date_data(date_data)

# Location now
# Slice and reshape the categorical data (location_code) for encoding
latitude_data = np.array(feature_list)[:, :, 0].reshape(-1, 1)
longitude_data = np.array(feature_list)[:, :, 1].reshape(-1, 1)

latitude_data = sequence_categorical_data(latitude_data)
longitude_data = sequence_categorical_data(longitude_data)

# Encode all categories
encoded_latitude_data, latitude_dict = process_category(latitude_data)
encoded_longitude_data, longitude_dict = process_category(longitude_data)
encoded_month_data, month_dict = process_category(month_data)
encoded_day_data, day_dict = process_category(day_data)
encoded_hour_data, hour_dict = process_category(hour_data)

file_names = {'latitude_data': latitude_dict, 'longitude_data': longitude_dict, 'month_data': month_dict, 'day_data': day_dict, 'hour_data': hour_dict}

Extracting and encoding categories


In [22]:
# Save the category dictionaries as json
for key, value in file_names.items():
    print(f'Saving {key} dictionary')
    with open(f"{TRAINING_DATA_FOLDER}/processed/{key}.json", 'w') as f:
        json.dump(value, f, indent=4)

# Print the shapes
print(np.shape(feature_list))
print(np.shape(target_list))
print(np.shape(encoded_latitude_data))
print(np.shape(encoded_longitude_data))
print(np.shape(encoded_month_data))
print(np.shape(encoded_day_data))
print(np.shape(encoded_hour_data))

Saving latitude_data dictionary
Saving longitude_data dictionary
Saving month_data dictionary
Saving day_data dictionary
Saving hour_data dictionary
(77858, 24, 4)
(77858, 24)
(77858, 52)
(77858, 52)
(77858, 12)
(77858, 31)
(77858, 9)


In [23]:
print('Data processing complete')
print('Splitting data')
# Remove the first, second and fourth columns from the feature list
feature_list = np.delete(feature_list, 0, axis=2)
feature_list = np.delete(feature_list, 0, axis=2)
feature_list = np.delete(feature_list, 0, axis=2)

# Call the function and unpack the returned dictionary
splits = split_data(feature_list, target_list, encoded_latitude_data, encoded_longitude_data, encoded_month_data, encoded_day_data, encoded_hour_data, test_size=0.2, validation_size=0.15, random_state=42)

# Unpack training, validation, and test sets from the dictionary
X_train, y_train, lat_train, lon_train, month_train, day_train, hour_train = splits["train"]
X_validate, y_validate, lat_validate, lon_validate, month_validate, day_validate, hour_validate = splits["validate"]
X_test, y_test, lat_test, lon_test, month_test, day_test, hour_test = splits["test"]
    
print('Scaling data')

# Scale the data
X_train_scaled, X_test_scaled, X_validate_scaled, y_train_scaled, y_test_scaled, y_validate_scaled = scale_data(X_train, X_test, X_validate, y_train, y_test, y_validate)

Data processing complete
Splitting data
Scaling data


In [24]:
# Save the data
print('Saving data')

final_data = [X_train_scaled, X_test_scaled, X_validate_scaled,
        y_train_scaled, y_test_scaled, y_validate_scaled,
        lat_train, lat_test, lat_validate,
        lon_train, lon_test, lon_validate,
        month_train, month_test, month_validate,
        day_train, day_test, day_validate,
        hour_train, hour_test, hour_validate]

data_names = ['X', 'y', 'lat', 'lon', 'month', 'day', 'hour']

# Save the data
save_data(f"{TRAINING_DATA_FOLDER}/processed/", final_data, data_names)
np.save(os.path.join(f"{TRAINING_DATA_FOLDER}/processed/", 'X_train_unscaled.npy'), X_train) # To reverse scaling for predictions
np.save(os.path.join(f"{TRAINING_DATA_FOLDER}/processed/", 'y_train_unscaled.npy'), y_train) # To reverse scaling for predictions
print('Data processing complete')

# Delete feature, target, and date data
os.remove(f'{TRAINING_DATA_FOLDER}/feature_data.npy')
os.remove(f'{TRAINING_DATA_FOLDER}/target_data.npy')
os.remove(f'{TRAINING_DATA_FOLDER}/date_data.npy')

Saving data
Data processing complete
